# Fonction Upper Bound 

On divise nos variables en clusters et en ordre, puis on realise une énumération pour chaque cluster (32 possibilités pour chaque cluster).

In [1]:
function compute_first_ub(c::Vector{Float64}, Q::Matrix{Float64}, a::Vector{Float64}, b::Float64)
    n = length(c)
    cluster_size = 5
    nb_clusters = div(n, cluster_size)
    UB = 0.0

    for cluster_id in 0:(nb_clusters-1)
        # Id du cluster
        idx = (cluster_id * cluster_size + 1):(cluster_id * cluster_size + cluster_size) # Liste d'indices du cluster donné
        c_cluster = c[idx]
        Q_cluster = Q[idx, idx]
        a_cluster = a[idx]  

        best_val = 0 #-Inf # - Infini mieux au cas ou on a des valeurs négatives

        # Enumérer les 32 combinaisons binaires
        for k in 0:31
            x = reverse(digits(k, base=2, pad=5))  # vector binair
            x_vec = collect(x)                     # convertir le vecteur en Array

            # Vérification des contraintes
            total_weight = 0
            for i in 1:cluster_size
                total_weight = total_weight + a_cluster[i] * x_vec[i]
            end
            if total_weight <= b
                # Evaluer la fonction objectif (linéal + quadratique)
                val = 0
                for i in 1:cluster_size
                    val = val + c_cluster[i] * x_vec[i] # Somme (c_i * x_i)
                end
                 for i in 1:cluster_size-1
                    for j in (i+1):cluster_size
                        val += Q_cluster[i, j] * x_vec[i] * x_vec[j] # Somme (c_ij * x_i * x_j)
                    end
                end

                best_val = max(best_val, val)
            end
        end

        UB += best_val
    end

    return UB
end



compute_first_ub (generic function with 1 method)

### Exemples

In [2]:
c = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0 , 7.0, 8.0, 9.0, 10.0]

Q = zeros(10, 10)
Q[1,2] = Q[2,1] = 1
Q[2,3] = Q[3,2] = 1
Q[3,4] = Q[4,3] = 1
Q[4,5] = Q[5,4] = 1
Q[6,7] = Q[7,6] = 2
Q[7,8] = Q[8,7] = 2
Q[8,9] = Q[9,8] = 2
Q[9,10] = Q[10,9] = 2

a = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

b = 5.0

ub = compute_first_ub(c, Q, a, b)
println("Resultat espéré: 67")
println("Resultat: ", ub)

Resultat espéré: 67
Resultat: 67.0


Énumération a la main, en divisan les variables en clusters:

In [3]:
c1 = [1, 2, 3, 4, 5]
a1 = [1, 1, 1, 1, 1]
Q1 = zeros(5, 5)
Q1[1,2] = Q1[2,1] = 1
Q1[2,3] = Q1[3,2] = 1
Q1[3,4] = Q1[4,3] = 1
Q1[4,5] = Q1[5,4] = 1
b = 5

best_val1 = -Inf
best_sol1 = []

for k in 0:31
    x = reverse(digits(k, base=2, pad=5))
    x_vec = collect(x)

    weight = sum(a1[i] * x_vec[i] for i in 1:5)
    if weight <= b
        val = sum(c1[i] * x_vec[i] for i in 1:5)
        for i in 1:4, j in (i+1):5
            val += Q1[i,j] * x_vec[i] * x_vec[j]
        end

        if val > best_val1
            best_val1 = val
            best_sol1 = copy(x_vec)
        end
    end
end

println("CLUSTER 1")
println("  Meilleur solution: ", best_sol1)
println("  Valeur totale: ", best_val1)

c2 = [6, 7, 8, 9, 10]
a2 = [1, 1, 1, 1, 1]
Q2 = zeros(5, 5)
Q2[1,2] = Q2[2,1] = 2
Q2[2,3] = Q2[3,2] = 2
Q2[3,4] = Q2[4,3] = 2
Q2[4,5] = Q2[5,4] = 2

best_val2 = -Inf
best_sol2 = []

for k in 0:31
    x = reverse(digits(k, base=2, pad=5))
    x_vec = collect(x)

    weight = sum(a2[i] * x_vec[i] for i in 1:5)
    if weight <= b
        val = sum(c2[i] * x_vec[i] for i in 1:5)
        for i in 1:4, j in (i+1):5
            val += Q2[i,j] * x_vec[i] * x_vec[j]
        end

        if val > best_val2
            best_val2 = val
            best_sol2 = copy(x_vec)
        end
    end
end

println("\nCLUSTER 2")
println("  Meilleur solution : ", best_sol2)
println("  Valeur totale: ", best_val2)

println("\nUB total = ", best_val1 + best_val2)



CLUSTER 1
  Meilleur solution: [1, 1, 1, 1, 1]
  Valeur totale: 19.0

CLUSTER 2
  Meilleur solution : [1, 1, 1, 1, 1]
  Valeur totale: 48.0

UB total = 67.0


### Dual Fonction

Évalution de la Fonction L_k

In [4]:
"""
Partitionne `n` variables en clusters de taille au plus `cluster_size`.
Retourne un vecteur de clusters, chacun un vecteur d'indices à base 1.
"""
function create_clusters(n::Int, cluster_size::Int=5)
    return [collect(i:min(i+cluster_size-1, n)) for i in 1:cluster_size:n]
end

function eval_Lk(
    C::Matrix{Float64}, c::Vector{Float64},
    lambda_kj::Dict{Tuple{Int,Int},Float64},
    mu_ij::Dict{Tuple{Int,Int},Float64},
    Ik::Vector{Int}, Jk::Vector{Int},
    xIk::Vector{Int}, yk::Vector{Float64},
    k::Int
)
    # 1) Termes linéaires et de contrainte de copie
    term1 = 0.0
    for (idx,i) in enumerate(Ik)
        # c_i * x_i
        term1 += c[i] * xIk[idx]
        # somme sur h≠k de λ_{h,i} * x_i  ==> λ indexé par (h,i), ici somme sur tous (h,i)
        for ((h,j), lam) in lambda_kj
            if j==i && h != k
                term1 += lam * xIk[idx]
            end
        end
    end
    # soustrait ∑_{j∈Jk} λ_{k,j} * y_j
    term1 -= sum(lambda_kj[(k,j)] * yv for (j,yv) in zip(Jk, yk))

    # 2) Terme quadratique intra-cluster : ½ ∑_{i,i'∈Ik, i≠i'} c_{ii'} x_i x_{i'}
    term2 = 0.0
    m = length(Ik)
    for p in 1:m
        for q in p+1:m
            term2 += 0.5 * C[Ik[p], Ik[q]] * xIk[p] * xIk[q]
        end
    end

    # 3) Termes croisés entre x et y
    term3 = 0.0
    for (p,i) in enumerate(Ik)
        for (q,j) in enumerate(Jk)
            coef = 0.5 * C[i,j]
            # ajoute ou soustrait mu selon que j>i ou j<i
            if j > i
                coef += get(mu_ij, (i,j), 0.0)
            else
                coef -= get(mu_ij, (i,j), 0.0)
            end
            term3 += coef * xIk[p] * yk[q]
        end
    end

    # Somme de tous les termes
    return term1 + term2 + term3
end

eval_Lk (generic function with 1 method)

Fonction duale finale

In [5]:
function w_lambda_mu(
    C::Matrix{Float64}, c::Vector{Float64}, a::Vector{Float64}, b::Float64,
    cluster_size::Int,
    lambda_kj::Dict{Tuple{Int,Int},Float64},
    mu_ij::Dict{Tuple{Int,Int},Float64}
)
    n = length(c)
    clusters = create_clusters(n, cluster_size)
    w_val = 0.0

    # Parcours de chaque cluster pour sommer les L_k maximales
    for (k, Ik) in enumerate(clusters)
        Jk = setdiff(1:n, Ik)
        best_Lk = -Inf
        m = length(Ik)

        # Énumère toutes les assignations binaires pour les variables complicationnelles xIk
        for s in 0:(2^m - 1)
            # Construction de xIk
            xIk = [Int((s >> (i-1)) & 1) for i in 1:m]
            # Capacité utilisée par xIk
            used_w = sum(a[Ik[i]] * xIk[i] for i in 1:m)
            rem_cap = b - used_w
            if rem_cap < 0
                continue
            end

            # Résolution gloutonne de la relaxation continue pour y_j ∈ Jk
            profits = Float64[]
            weights = Float64[]
            for j in Jk
                inter = sum(
                    0.5 * C[Ik[i], j] * xIk[i] +
                    get(mu_ij, (Ik[i], j), 0.0) * xIk[i]
                    for i in 1:m
                )
                lam = get(lambda_kj, (k, j), 0.0)
                push!(profits, inter - lam)
                push!(weights, a[j])
            end

            # Knapsack fractionnaire
            ratios = profits ./ weights
            idx = sortperm(ratios, rev=true)
            yk = zeros(Float64, length(Jk))
            cap = rem_cap
            for j_idx in idx
                if cap <= 0
                    break
                end
                take = min(1.0, cap / weights[j_idx])
                yk[j_idx] = take
                cap -= weights[j_idx] * take
            end

            # Évalue L_k pour ce couple (xIk, yk)
            Lk_val = eval_Lk(C, c, lambda_kj, mu_ij, Ik, Jk, xIk, yk, k)
            best_Lk = max(best_Lk, Lk_val)
        end

        # Ajoute la meilleure valeur trouvée pour ce cluster
        w_val += best_Lk
    end

    return w_val
end

w_lambda_mu (generic function with 1 method)

Deuxieme version de w_lambda_mu(....), qui retourne le meilleur couple (x,y)

In [6]:
function solve_dual_primal(
    C::Matrix{Float64}, c::Vector{Float64}, a::Vector{Float64}, b::Float64,
    cluster_size::Int,
    lambda_kj::Dict{Tuple{Int,Int},Float64},
    mu_ij::Dict{Tuple{Int,Int},Float64}
)
    n = length(c)
    clusters = create_clusters(n, cluster_size)
    w_val = 0.0
    xstar = Vector{Vector{Int}}(undef, length(clusters))
    ystar = Vector{Vector{Float64}}(undef, length(clusters))

    for (k, Ik) in enumerate(clusters)
        Jk = setdiff(1:n, Ik)
        m = length(Ik)
        best_Lk = -Inf
        best_x = zeros(Int, m)
        best_y = zeros(Float64, length(Jk))

        for s in 0:(2^m - 1)
            xIk = [ (s >> (i-1)) & 1 for i in 1:m ]
            used_w = sum(a[Ik[i]] * xIk[i] for i in 1:m)
            rem_cap = b - used_w
            if rem_cap < 0
                continue
            end
            profits = Float64[]; weights = Float64[]
            for j in Jk
                inter = sum(
                    0.5 * C[Ik[i], j] * xIk[i] +
                    get(mu_ij, (Ik[i], j), 0.0) * xIk[i]
                    for i in 1:m
                )
                lam = get(lambda_kj, (k, j), 0.0)
                push!(profits, inter - lam)
                push!(weights, a[j])
            end
            ratios = profits ./ weights
            idx = sortperm(ratios, rev=true)
            yk = zeros(Float64, length(Jk)); cap = rem_cap
            for j_idx in idx
                cap <= 0 && break
                take = min(1.0, cap / weights[j_idx])
                yk[j_idx] = take
                cap -= weights[j_idx] * take
            end
            val = eval_Lk(C, c, lambda_kj, mu_ij, Ik, Jk, xIk, yk, k)
            if val > best_Lk
                best_Lk = val
                best_x .= xIk
                best_y .= yk
            end
        end
        w_val += best_Lk
        xstar[k] = copy(best_x)
        ystar[k] = copy(best_y)
    end

    return (w_val=w_val, xstar=xstar, ystar=ystar)
end

solve_dual_primal (generic function with 1 method)

## Implémentation d'algorithmes de gradients

In [7]:
function subgradient_solve(
    C::Matrix{Float64}, c::Vector{Float64}, a::Vector{Float64}, b::Float64,
    cluster_size::Int,
    lambda_init::Dict{Tuple{Int,Int},Float64},
    mu_init::Dict{Tuple{Int,Int},Float64};
    maxiter::Int=100,
    alpha0::Float64=1.0,
    p::Float64=0.9
)
    # Copie des valeurs initiales
    lambda = deepcopy(lambda_init)
    mu = deepcopy(mu_init)

    best_dual = +Inf
    best_lambda, best_mu = lambda, mu

    alpha = alpha0

    for t in 1:maxiter
        # 1) Évalue la fonction duale et récupère les argmax primaux (x*, y*)
        sol = solve_dual_primal(C, c, a, b, cluster_size, lambda, mu)
        wval = sol.w_val

        # 2) Mémorise la meilleure valeur duale
        if wval < best_dual
            best_dual = wval
            best_lambda, best_mu = deepcopy(lambda), deepcopy(mu)
        end

        # 3) Calcul des sous-gradients pour λ et μ
        glambda = Dict{Tuple{Int,Int},Float64}()
        gmu = Dict{Tuple{Int,Int},Float64}()

        for (k, Ik) in enumerate(create_clusters(length(c), cluster_size))
            Jk = setdiff(1:length(c), Ik)
            xIk = sol.xstar[k]
            yk  = sol.ystar[k]
            for (idx,j) in enumerate(Jk)
                kj = cluster_of[j]
                pj = pos_in_cluster[j]
                xj = sol.xstar[kj][pj] 
                # g_lambda[k,j] = x_j* - y_j^k*
                glambda[(k,j)] = xj - yk[idx]
            end
        end

        # Sous-gradients pour contraintes croisées
        for ((i,j), _) in mu
            # Suppose on dispose de cluster_of et pos_in_cluster
            k_i, pos_i = cluster_of[i], pos_in_cluster[i]
            k_j, pos_j = cluster_of[j], pos_in_cluster[j]
            x_i = sol.xstar[k_i][pos_i]
            x_j = sol.xstar[k_j][pos_j]
            y_ji = sol.ystar[k_j][pos_i]
            y_ij = sol.ystar[k_i][pos_j]
            # g_mu[i,j] = x_i*y_ji - x_j*y_ij
            gmu[(i,j)] = x_i*y_ji - x_j*y_ij
        end

        # 4) Mise à jour des multiplicateurs
        for key in keys(lambda)
            lambda[key] = max(0.0, lambda[key] - alpha * glambda[key])
        end
        for key in keys(mu)
            mu[key] = mu[key] - alpha * gmu[key]
        end

        # 5) Réduction du pas
        alpha *= p
    end

    return best_lambda, best_mu, best_dual
end

subgradient_solve (generic function with 1 method)

Test 1:

In [8]:
c = [1.0, 2.0 ,3.0 ,4.0 ,5.0 , 6.0 ,7.0 ,8.0 ,9.0 ,10.0]
Q = zeros(10,10)
Q[1,2] = Q[2,1] = 1
Q[2,3] = Q[3,2] = 1
Q[3,4] = Q[4,3] = 1
Q[4,5] = Q[5,4] = 1
Q[6,7] = Q[7,6] = 2
Q[7,8] = Q[8,7] = 2
Q[8,9] = Q[9,8] = 2
Q[9,10]= Q[10,9]= 2

a = fill(2.0, 10)
cluster_size = 5

# 2) Para cada cluster, elegimos las dos primeras variables y las ponemos a 1
clusters = create_clusters(10, 5)
for Ik in clusters
    # por ejemplo, fijamos Ik[1] e Ik[2] a peso 1
    a[ Ik[1] ] = 1
    a[ Ik[2] ] = 1
end


b = 4.0

cluster_of = Dict{Int,Int}()
pos_in_cluster = Dict{Int,Int}()
for (k, Ik) in enumerate(clusters)
    for (pos, i) in enumerate(Ik)
        cluster_of[i] = k
        pos_in_cluster[i] = pos
    end
end

lambda_kj = Dict{Tuple{Int,Int},Float64}()
for (k, Ik) in enumerate(clusters)
    for j in setdiff(1:length(c), Ik)
        lambda_kj[(k,j)] = 0.0
    end
end

# initialize mu_{i,j} = 0 for every cross-cluster i<j
mu_ij = Dict{Tuple{Int,Int},Float64}()
for i in 1:length(c)-1, j in i+1:length(c)
    if cluster_of[i] != cluster_of[j]
        mu_ij[(i,j)] = 0.0
    end
end


In [9]:
lamda_opt, mu_opt, dual_val = subgradient_solve(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 50,
    alpha0 = 1.0,
    p      = 0.9
)

println("Best dual bound = ", dual_val)
ub = compute_first_ub(c, Q, a, b)
println("Upper bound initiale: ", ub)

Best dual bound = 33.5
Upper bound initiale: 35.0


### Versions de l'algo de sous gradient

Méthode 2: Série convergente


In [14]:
function subgradient_solve_m2(
    C::Matrix{Float64}, c::Vector{Float64}, a::Vector{Float64}, b::Float64,
    cluster_size::Int,
    lambda_init::Dict{Tuple{Int,Int},Float64},
    mu_init::Dict{Tuple{Int,Int},Float64};
    maxiter::Int=100,
    alpha0::Float64=1.0,
    p::Float64=0.9
)
    # Copie des valeurs initiales
    lambda = deepcopy(lambda_init)
    mu = deepcopy(mu_init)

    best_dual = +Inf
    best_lambda, best_mu = lambda, mu

    alpha = alpha0

    for t in 1:maxiter
        # 1) Évalue la fonction duale et récupère les argmax primaux (x*, y*)
        sol = solve_dual_primal(C, c, a, b, cluster_size, lambda, mu)
        wval = sol.w_val

        # 2) Mémorise la meilleure valeur duale
        if wval < best_dual
            best_dual = wval
            best_lambda, best_mu = deepcopy(lambda), deepcopy(mu)
        end

        # 3) Calcul des sous-gradients pour λ et μ
        glambda = Dict{Tuple{Int,Int},Float64}()
        gmu = Dict{Tuple{Int,Int},Float64}()

        for (k, Ik) in enumerate(create_clusters(length(c), cluster_size))
            Jk = setdiff(1:length(c), Ik)
            xIk = sol.xstar[k]
            yk  = sol.ystar[k]
            for (idx,j) in enumerate(Jk)
                kj = cluster_of[j]
                pj = pos_in_cluster[j]
                xj = sol.xstar[kj][pj] 
                # g_lambda[k,j] = x_j* - y_j^k*
                glambda[(k,j)] = xj - yk[idx]
            end
        end

        # Sous-gradients pour contraintes croisées
        for ((i,j), _) in mu
            # Suppose on dispose de cluster_of et pos_in_cluster
            k_i, pos_i = cluster_of[i], pos_in_cluster[i]
            k_j, pos_j = cluster_of[j], pos_in_cluster[j]
            x_i = sol.xstar[k_i][pos_i]
            x_j = sol.xstar[k_j][pos_j]
            y_ji = sol.ystar[k_j][pos_i]
            y_ij = sol.ystar[k_i][pos_j]
            # g_mu[i,j] = x_i*y_ji - x_j*y_ij
            gmu[(i,j)] = x_i*y_ji - x_j*y_ij
        end

        # 4) Mise à jour des multiplicateurs
        for key in keys(lambda)
            lambda[key] = max(0.0, lambda[key] - alpha * glambda[key])
        end
        for key in keys(mu)
            mu[key] = mu[key] - alpha * gmu[key]
        end

        # 5) Réduction du pas
        alpha =  alpha0 * p^(t-1)
    end

    return best_lambda, best_mu, best_dual
end


subgradient_solve_m2 (generic function with 1 method)

Méthode 3: Relaxation

In [15]:
function subgradient_solve_m3(
    C::Matrix{Float64}, c::Vector{Float64}, a::Vector{Float64}, b::Float64,
    cluster_size::Int,
    lambda_init::Dict{Tuple{Int,Int},Float64},
    mu_init::Dict{Tuple{Int,Int},Float64};
    fbar::Float64,
    maxiter::Int=100,
    p::Float64=0.9
)
    # Copie des valeurs initiales
    lambda = deepcopy(lambda_init)
    mu = deepcopy(mu_init)

    best_dual = +Inf
    best_lambda, best_mu = lambda, mu

    alpha = 1.0

    for t in 1:maxiter
        # 1) Évalue la fonction duale et récupère les argmax primaux (x*, y*)
        sol = solve_dual_primal(C, c, a, b, cluster_size, lambda, mu)
        wval = sol.w_val

        # 2) Mémorise la meilleure valeur duale
        if wval < best_dual
            best_dual = wval
            best_lambda, best_mu = deepcopy(lambda), deepcopy(mu)
        end

        # 3) Calcul des sous-gradients pour λ et μ

        g2 = 0.0
        glambda = Dict{Tuple{Int,Int},Float64}()
        gmu = Dict{Tuple{Int,Int},Float64}()

        for (k, Ik) in enumerate(create_clusters(length(c), cluster_size))
            Jk = setdiff(1:length(c), Ik)
            xIk = sol.xstar[k]
            yk  = sol.ystar[k]
            for (idx,j) in enumerate(Jk)
                kj = cluster_of[j]
                pj = pos_in_cluster[j]
                xj = sol.xstar[kj][pj] 
                # g_lambda[k,j] = x_j* - y_j^k*
                val = xj - yk[idx]
                glambda[(k,j)] = val
                g2 += val^2
            end
        end

        # Sous-gradients pour contraintes croisées
        for ((i,j), _) in mu
            # Suppose on dispose de cluster_of et pos_in_cluster
            k_i, pos_i = cluster_of[i], pos_in_cluster[i]
            k_j, pos_j = cluster_of[j], pos_in_cluster[j]
            x_i = sol.xstar[k_i][pos_i]
            x_j = sol.xstar[k_j][pos_j]
            y_ji = sol.ystar[k_j][pos_i]
            y_ij = sol.ystar[k_i][pos_j]
            # g_mu[i,j] = x_i*y_ji - x_j*y_ij
            val = x_i*y_ji - x_j*y_ij
            gmu[(i,j)] = val
            g2 += val^2
        end

        # 4) Mise à jour des multiplicateurs
        for key in keys(lambda)
            lambda[key] = max(0.0, lambda[key] - alpha * glambda[key])
        end
        for key in keys(mu)
            mu[key] = mu[key] - alpha * gmu[key]
        end

        # 5) Réduction du pas - with safeguard
        if wval > fbar
            alpha = p * (wval - fbar) / sqrt(g2 + eps())
        else
            alpha = p * alpha  # Just reduce the current step size
        end
    end

    return best_lambda, best_mu, best_dual
end

subgradient_solve_m3 (generic function with 1 method)

In [16]:
lamda_opt_m2, mu_opt_m2, dual_val_m2 = subgradient_solve_m2(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 50,
    alpha0 = 1.0,
    p      = 0.9
)

println("Best dual bound (méthode 2) = ", dual_val_m2)

Best dual bound (méthode 2) = 33.5


In [17]:
"""
  greedy_linear(c, Q, a, b)

Heuristique O(n log n) :
- Tri des items par ratio c[i]/a[i] décroissant
- Remplissage glouton
- Évaluation complète de fbar
Retourne (X::Vector{Bool}, fbar::Float64).
"""
function greedy_linear(c::Vector{Float64},
                       Q::Matrix{Float64},
                       a::Vector{Float64},
                       b::Float64)
    n = length(c)
    order = sortperm(c ./ a, rev=true)  # indices triés par ratio
    X = falses(n)
    cap = b

    # Remplissage
    for idx in order
        if a[idx] <= cap
            X[idx] = true
            cap -= a[idx]
        end
    end

    # Calcul de fbar complet
    fbar = 0.0
    for i in 1:n
        if X[i]
            fbar += c[i]
            for j in (i+1):n
                fbar += Q[i,j] * (X[j] ? 1.0 : 0.0)
            end
        end
    end

    return X, fbar
end

"""
  greedy_plus_oneflip(c, Q, a, b)

Heuristique :
1) greedy_linear  
2) Essaye un seul flip 0→1 si gain positif  
Retourne (X::Vector{Bool}, fbar::Float64).
"""
function greedy_plus_oneflip(c::Vector{Float64},
                             Q::Matrix{Float64},
                             a::Vector{Float64},
                             b::Float64)
    n = length(c)
    X, fbar = greedy_linear(c, Q, a, b)

    # Capacité restante
    cap = b - sum(a[ii] for ii in 1:n if X[ii])

    # Un seul flip
    for idx in 1:n
        if !X[idx] && a[idx] <= cap
            # gain marginal delta si on ajoute idx
            delta = c[idx]
            for k in 1:n
                if X[k]
                    delta += Q[idx,k]
                end
            end
            if delta > 0
                X[idx] = true
                fbar += delta
                break
            end
        end
    end

    return X, fbar
end

X0, fbar = greedy_linear(c, Q, a, b)
println("greedy_linear fbar = ", fbar)

X1, fbar1 = greedy_plus_oneflip(c, Q, a, b)
println("greedy_plus_oneflip fbar = ", fbar1)


greedy_linear fbar = 25.0
greedy_plus_oneflip fbar = 25.0


Heuristique gourmande ( Eric)

In [18]:
"""
  greedy_heuristic(c, Q, a, b)
Heuristique gourmande basée sur l'algorithme du document :
- Initialise tous les articles dans le sac
- Retire itérativement l'article avec le plus petit ratio c_j tant que la contrainte de capacité est violée
- Calcule les ratios c_j pour tous les articles restants hors du sac
- Ajoute l'article avec le meilleur ratio si c_j > 0
Retourne (X::Vector{Bool}, fbar::Float64).
"""
function greedy_heuristic(c::Vector{Float64},
                         Q::Matrix{Float64},
                         a::Vector{Float64},
                         b::Float64)
    # Create a copy to avoid modifying the original
    c_work = copy(c)
    n = length(c)
    
    # Étape 1: Mettre tous les articles dans le sac
    # K1 = ensemble des articles dans le sac
    # K0 = ensemble des articles hors du sac
    K1 = Set(1:n)  # Tous les articles initialement dans le sac
    K0 = Set{Int}()  # Aucun article hors du sac
    
    # Tant que la contrainte de capacité est violée
    while sum(a[j] for j in K1) > b
        # Trouver k ∈ K1 tel que c_k = min c_j pour j ∈ K1
        c_values = [c_work[j] for j in K1]
        min_c = minimum(c_values)
        k = first(j for j in K1 if c_work[j] == min_c)
        
        # Mise à jour: K1 ← K1 \ {k}, K0 ← K0 ∪ {k}
        delete!(K1, k)
        push!(K0, k)
        
        # Mise à jour de c_j pour j ∈ K1
        for j in K1
            c_work[j] = c_work[j] - Q[j,k]
        end
    end
    
    # Étape 2: Amélioration
    # Pour tout j ∈ K0, calculer c_j et ajouter si c_j > 0
    for j in K0
        if j <= length(a)  # Vérification de sécurité
            # Calculer c_j = c_j - sum(Q[j,k] for k ∈ K1)
            c_j = c_work[j]
            for k in K1
                c_j -= Q[j,k]
            end
            
            # Si c_j > 0, ajouter j au sac
            if c_j > 0
                push!(K1, j)
                delete!(K0, j)
            end
        end
    end
    
    # Construction du vecteur solution
    X = falses(n)
    for j in K1
        X[j] = true
    end
    
    # Calcul de la fonction objectif
    fbar = 0.0
    for i in 1:n
        if X[i]
            fbar += c[i]  # Use original c for objective calculation
            for j in (i+1):n
                if X[j]
                    fbar += Q[i,j]
                end
            end
        end
    end
    
    return X, fbar
end

greedy_heuristic

In [19]:
X_greedy, fbar_greedy = greedy_heuristic(c, Q, a, b)
println("greedy_heuristic fbar = ", fbar_greedy)

greedy_heuristic fbar = 60.0


Heuristique : Recuit-Simulé

In [20]:
"""
  simulated_annealing(c, Q, a, b, x_init; tau_init=100.0, rho=0.95, nbpaliers=50, L=100)

Méthode heuristique du recuit-simulé pour le problème de sac-à-dos quadratique.
Algorithme basé sur la description du document.

Arguments:
- c: vecteur des coefficients linéaires
- Q: matrice des coefficients quadratiques
- a: vecteur des poids des articles
- b: capacité du sac
- x_init: solution initiale (Vector{Bool})
- tau_init: température initiale (défaut: 100.0)
- rho: facteur de réduction de température (défaut: 0.95)
- nbpaliers: nombre de paliers de température (défaut: 50)
- L: nombre d'itérations par palier (défaut: 100)

Retourne (X::Vector{Bool}, fbar::Float64).
"""
function simulated_annealing(c::Vector{Float64},
                           Q::Matrix{Float64},
                           a::Vector{Float64},
                           b::Float64,
                           x_init::BitVector;
                           tau_init::Float64=100.0,
                           rho::Float64=0.95,
                           nbpaliers::Int=50,
                           L::Int=100)
    c_work = copy(c)
    n = length(c_work)
    # Fonction pour calculer la valeur objective
    function objective_value(x::BitVector)
        fval = 0.0
        for i in 1:n
            if x[i]
                fval += c_work[i]
                for j in (i+1):n
                    if x[j]
                        fval += Q[i,j]
                    end
                end
            end
        end
        return fval
    end
    
    # Fonction pour vérifier la faisabilité
    function is_feasible(x::BitVector)
        return sum(a[i] for i in 1:n if x[i]) <= b
    end
    
    # Fonction pour générer un voisin en échangeant deux articles
    function generate_neighbor(x::BitVector)
        x_new = copy(x)
        
        # Trouver un article dans le sac et un article hors du sac
        in_sac = findall(x)
        out_sac = findall(.!x)
        
        if length(in_sac) == 0 || length(out_sac) == 0
            return x_new  # Pas de voisin possible
        end
        
        # Choisir aléatoirement un article de chaque ensemble
        idx_in = rand(in_sac)
        idx_out = rand(out_sac)
        
        # Échanger: sortir idx_in, entrer idx_out
        x_new[idx_in] = false
        x_new[idx_out] = true
        
        return x_new
    end
    
    # (1) x ← solution initiale
    x = copy(x_init)
    best_x = copy(x)
    best_f = objective_value(x)
    
    # (2) τ ← température initiale
    tau = tau_init
    
    # (3) faire nbpaliers fois
    for palier in 1:nbpaliers
        
        # (3.1) faire L fois
        for iteration in 1:L
            
            # Tirer au hasard y voisin de x
            y = generate_neighbor(x)
            
            # Vérifier la faisabilité
            if !is_feasible(y)
                continue  # Rejeter les solutions non faisables
            end
            
            # Δ ← f(x) - f(y)
            f_x = objective_value(x)
            f_y = objective_value(y)
            delta = f_x - f_y  # Pour un problème de maximisation
            
            # si Δ ≤ 0 alors (* amélioration *)
            if delta <= 0
                x = copy(y)
                # Mettre à jour la meilleure solution
                if f_y > best_f
                    best_x = copy(y)
                    best_f = f_y
                end
            else
                # sinon (* détérioration *)
                # tirer au hasard p ∈ [0,1]
                p = rand()
                
                # proba := e^(-Δ/τ)
                proba = exp(-delta / tau)
                
                # si (proba > p) alors x ← y
                if proba > p
                    x = copy(y)
                end
            end
        end
        
        # (3.2) τ ← ρ * τ
        tau = rho * tau
    end
    
    # (4) retourner x (* x est la meilleure solution trouvée *)
    return best_x, best_f
end

simulated_annealing

In [21]:
x_init = rand(Bool, length(c))  # Random initial solution
x_init

10-element Vector{Bool}:
 1
 0
 1
 0
 1
 1
 0
 0
 0
 0

In [22]:
X_sa, fbar_sa = simulated_annealing(c, Q, a, b, X_greedy)
X_greedy, fbar_greedy = greedy_heuristic(c, Q, a, b)

println("Simulated Annealing fbar = ", fbar_sa)

println("greedy_heuristic fbar = ", fbar_greedy)

Simulated Annealing fbar = 60.0
greedy_heuristic fbar = 60.0


In [23]:
lamda_opt_m3, mu_opt_m3, dual_val_m3 = subgradient_solve_m3(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    fbar,
    maxiter = 50,
    p      = 0.9
)

println("Best dual bound (méthode 3) = ", dual_val_m3)

Best dual bound (méthode 3) = 33.5


In [24]:
println("Upper bound initiale: ", ub)
println("Dual bound (méthode 2) = ", dual_val_m2)
println("Dual bound (méthode 3) = ", dual_val_m3)


Upper bound initiale: 35.0
Dual bound (méthode 2) = 33.5
Dual bound (méthode 3) = 33.5


In [25]:
using Random

# 1) Données du problème à 30 variables
n = 30
c = Float64.(1:n)            # [1.0, 2.0, …, 30.0]

# 2) Matrice de couplages Q en « chaînes par blocs »
Q = zeros(n, n)
for i in 1:n-1
    w = i ≤ 10  ? 1.0 :
        i ≤ 20  ? 2.0 :
                  3.0
    Q[i, i+1] = w
    Q[i+1, i] = w
end
# quelques croisements additionnels
Q[5,15]  = Q[15,5]  = 0.5
Q[10,20] = Q[20,10] = 0.7
Q[12,25] = Q[25,12] = 1.2

# 3) Poids a : tous à 2.0, puis on fixe 2 poids à 1 par cluster
a = fill(2.0, n)
cluster_size = 5

# 4) Création des clusters et choix aléatoire de deux indices à poids=1
clusters = create_clusters(n, cluster_size)
for Ik in clusters
    # por ejemplo, fijamos Ik[1] e Ik[2] a peso 1
    a[ Ik[1] ] = 1
    a[ Ik[2] ] = 1
end

# 5) Capacité b (par exemple 40% de la somme des a)
b = 0.4 * sum(a)

# 6) Tables de mapping pour les clusters
cluster_of     = Dict{Int,Int}()
pos_in_cluster = Dict{Int,Int}()
for (k, Ik) in enumerate(clusters)
    for (pos, i) in enumerate(Ik)
        cluster_of[i]     = k
        pos_in_cluster[i] = pos
    end
end

# 7) Initialisation des multiplicateurs λ_{k,j}
lambda_kj = Dict{Tuple{Int,Int},Float64}()
for (k, Ik) in enumerate(clusters)
    for j in setdiff(1:n, Ik)
        lambda_kj[(k,j)] = 0.0
    end
end

# 8) Initialisation des multiplicateurs μ_{i,j}
mu_ij = Dict{Tuple{Int,Int},Float64}()
for i in 1:n-1, j in i+1:n
    if cluster_of[i] != cluster_of[j]
        mu_ij[(i,j)] = 0.0
    end
end

# 9) Affichage de l’instance
println("n            = ", n)
println("cluster_size = ", cluster_size)
println("b            = ", b)
println("clusters     = ", clusters)
println("c            = ", c)
println("a            = ", a)


n            = 30
cluster_size = 5
b            = 19.200000000000003
clusters     = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25], [26, 27, 28, 29, 30]]
c            = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]
a            = [1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0]


In [26]:
X0, fbar = greedy_linear(c, Q, a, b)
println("greedy_linear fbar = ", fbar)

X1, fbar1 = greedy_plus_oneflip(c, Q, a, b)
println("greedy_plus_oneflip fbar = ", fbar1)

greedy_linear fbar = 330.2
greedy_plus_oneflip fbar = 330.2


In [27]:
lamda_opt_m2, mu_opt_m2, dual_val_m2 = subgradient_solve_m2(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 1,
    alpha0 = 1.0,
    p      = 0.9
)

println("Best dual bound (méthode 2) = ", dual_val_m2)

Best dual bound (méthode 2) = 500.4


In [28]:
fbar= 520.4

lamda_opt_m3, mu_opt_m3, dual_val_m3 = subgradient_solve_m3(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    fbar,
    maxiter = 50,
    p      = 0.9
)

println("Best dual bound (méthode 3) = ", dual_val_m3)

Best dual bound (méthode 3) = 500.4


### exemple 1: n = 1000

In [77]:
using Random


n = 1000
c = Float64.(1:n)            # [1.0, 2.0, …, 1000.0]

# matrice Q en « chaînes par blocs »
Q = zeros(n, n)
for i in 1:n-1
    w = i ≤ n÷3  ? 1.0 :
        i ≤ 2n÷3 ? 2.0 :
                   3.0
    Q[i, i+1] = w
    Q[i+1, i] = w
end
# quelques croisements additionnels fixes
Q[round(Int,n/4), round(Int,3n/4)] = 0.5
Q[round(Int,n/3), round(Int,n/2)]   = 0.7
Q[round(Int,n/2), round(Int,2n/3)]  = 1.2

# poids a et clusters
a = fill(2.0, n)
cluster_size = 5
clusters = create_clusters(n, cluster_size)
for Ik in clusters
    a[Ik[1]] = 1
    a[Ik[2]] = 1
end

b = 0.4 * sum(a)  # capacité

# tables de mapping
cluster_of     = Dict{Int,Int}()
pos_in_cluster = Dict{Int,Int}()
for (k, Ik) in enumerate(clusters)
    for (pos, i) in enumerate(Ik)
        cluster_of[i]     = k
        pos_in_cluster[i] = pos
    end
end

# λ_{k,j}
lambda_kj = Dict{Tuple{Int,Int},Float64}()
for (k, Ik) in enumerate(clusters)
    for j in setdiff(1:n, Ik)
        lambda_kj[(k,j)] = 0.0
    end
end

# μ_{i,j}
mu_ij = Dict{Tuple{Int,Int},Float64}()
for i in 1:n-1, j in i+1:n
    if cluster_of[i] != cluster_of[j]
        mu_ij[(i,j)] = 0.0
    end
end

println("Instance 1000 vars  •  b = ", b)


Instance 1000 vars  •  b = 640.0


In [78]:

X0, fbar = greedy_linear(c, Q, a, b)
println("greedy_linear fbar = ", fbar)

greedy_linear fbar = 336530.0


In [79]:

X1, fbar1 = greedy_plus_oneflip(c, Q, a, b)
println("greedy_plus_oneflip fbar = ", fbar1)

greedy_plus_oneflip fbar = 336530.0


In [80]:
X_greedy, fbar_greedy = greedy_heuristic(c, Q, a, b)
println("greedy_heuristic fbar = ", fbar_greedy)

greedy_heuristic fbar = 502493.4


In [81]:
X_sa, fbar_sa = simulated_annealing(c, Q, a, b, X_greedy)
println("Simulated Annealing fbar = ", fbar_sa)


Simulated Annealing fbar = 502493.4


In [82]:
ub = compute_first_ub(c, Q, a, b)


502100.0

In [83]:
lamda_opt_m2, mu_opt_m2, dual_val_m2 = subgradient_solve_m2(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 50,
    alpha0 = 1.0,
    p      = 0.9
)

InterruptException: InterruptException:

In [84]:
lamda_opt_m3, mu_opt_m3, dual_val_m3 = subgradient_solve_m3(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    fbar = fbar_sa,
    maxiter = 50,
    p      = 0.9
)

InterruptException: InterruptException:

In [85]:
println("Upper bound initiale: ", ub)
println("Dual bound (méthode 2) = ", dual_val_m2)
println("Dual bound (méthode 3) = ", dual_val_m3)

Upper bound initiale: 502100.0
Dual bound (méthode 2) = 125849.2
Dual bound (méthode 3) = 125849.2


### exemple 2: n = 500

In [67]:
n = 500
c = Float64.(1:n)

Q = zeros(n, n)
for i in 1:n-1
    w = i ≤ n÷3  ? 1.0 :
        i ≤ 2n÷3 ? 2.0 :
                   3.0
    Q[i, i+1] = w
    Q[i+1, i] = w
end
Q[round(Int,n/4), round(Int,3n/4)] = 0.5
Q[round(Int,n/3), round(Int,n/2)]   = 0.7
Q[round(Int,n/2), round(Int,2n/3)]  = 1.2

a = fill(2.0, n)
cluster_size = 5
clusters = create_clusters(n, cluster_size)
for Ik in clusters
    a[Ik[1]] = 1
    a[Ik[2]] = 1
end

b = 0.4 * sum(a)

cluster_of     = Dict{Int,Int}()
pos_in_cluster = Dict{Int,Int}()
for (k, Ik) in enumerate(clusters)
    for (pos, i) in enumerate(Ik)
        cluster_of[i]     = k
        pos_in_cluster[i] = pos
    end
end

lambda_kj = Dict{Tuple{Int,Int},Float64}()
for (k, Ik) in enumerate(clusters)
    for j in setdiff(1:n, Ik)
        lambda_kj[(k,j)] = 0.0
    end
end

mu_ij = Dict{Tuple{Int,Int},Float64}()
for i in 1:n-1, j in i+1:n
    if cluster_of[i] != cluster_of[j]
        mu_ij[(i,j)] = 0.0
    end
end

println("Instance 500 vars   •  b = ", b)

Instance 500 vars   •  b = 320.0


In [68]:

X0, fbar = greedy_linear(c, Q, a, b)
println("greedy_linear fbar = ", fbar)

greedy_linear fbar = 84402.0


In [69]:

X1, fbar1 = greedy_plus_oneflip(c, Q, a, b)
println("greedy_plus_oneflip fbar = ", fbar1)

greedy_plus_oneflip fbar = 84402.0


In [70]:
X_greedy, fbar_greedy = greedy_heuristic(c, Q, a, b)
println("greedy_heuristic fbar = ", fbar_greedy)

greedy_heuristic fbar = 126246.4


In [71]:
X_sa, fbar_sa = simulated_annealing(c, Q, a, b, X_greedy)
println("Simulated Annealing fbar = ", fbar_sa)

Simulated Annealing fbar = 126246.4


In [72]:
ub = compute_first_ub(c, Q, a, b)

126050.0

In [73]:
lamda_opt_m2, mu_opt_m2, dual_val_m2 = subgradient_solve_m2(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 50,
    alpha0 = 1.0,
    p      = 0.9
)

(Dict((5, 499) => 0.0, (12, 418) => 0.0, (75, 444) => 0.0, (48, 61) => 0.0, (60, 171) => 0.0, (52, 32) => 0.0, (25, 73) => 0.0, (91, 30) => 0.0, (100, 10) => 0.0, (52, 193) => 0.0…), Dict((262, 419) => 0.0, (349, 464) => 0.0, (243, 313) => 0.0, (416, 469) => 0.0, (308, 347) => 0.0, (5, 499) => 0.0, (12, 418) => 0.0, (75, 444) => 0.0, (161, 462) => 0.0, (48, 61) => 0.0…), 125849.2)

In [74]:
lamda_opt_m3, mu_opt_m3, dual_val_m3 = subgradient_solve_m3(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    fbar = fbar_sa,
    maxiter = 50,
    p      = 0.9
)

(Dict((5, 499) => 0.0, (12, 418) => 0.0, (75, 444) => 0.0, (48, 61) => 0.0, (60, 171) => 0.0, (52, 32) => 0.0, (25, 73) => 0.0, (91, 30) => 0.0, (100, 10) => 0.0, (52, 193) => 0.0…), Dict((262, 419) => 0.0, (349, 464) => 0.0, (243, 313) => 0.0, (416, 469) => 0.0, (308, 347) => 0.0, (5, 499) => 0.0, (12, 418) => 0.0, (75, 444) => 0.0, (161, 462) => 0.0, (48, 61) => 0.0…), 125849.2)

In [76]:
println("Upper bound initiale: ", ub)
println("Dual bound (méthode 2) = ", dual_val_m2)
println("Dual bound (méthode 3) = ", dual_val_m3)

Upper bound initiale: 126050.0
Dual bound (méthode 2) = 125849.2
Dual bound (méthode 3) = 125849.2


### exemple 3: n = 100

In [56]:


n = 100
c = Float64.(1:n)

Q = zeros(n, n)
for i in 1:n-1
    w = i ≤ n÷3  ? 1.0 :
        i ≤ 2n÷3 ? 2.0 :
                   3.0
    Q[i, i+1] = w
    Q[i+1, i] = w
end
Q[round(Int,n/4), round(Int,3n/4)] = 0.5
Q[round(Int,n/3), round(Int,n/2)]   = 0.7
Q[round(Int,n/2), round(Int,2n/3)]  = 1.2

a = fill(2.0, n)
cluster_size = 5
clusters = create_clusters(n, cluster_size)
for Ik in clusters
    a[Ik[1]] = 1
    a[Ik[2]] = 1
end

b = 0.4 * sum(a)

cluster_of     = Dict{Int,Int}()
pos_in_cluster = Dict{Int,Int}()
for (k, Ik) in enumerate(clusters)
    for (pos, i) in enumerate(Ik)
        cluster_of[i]     = k
        pos_in_cluster[i] = pos
    end
end

lambda_kj = Dict{Tuple{Int,Int},Float64}()
for (k, Ik) in enumerate(clusters)
    for j in setdiff(1:n, Ik)
        lambda_kj[(k,j)] = 0.0
    end
end

mu_ij = Dict{Tuple{Int,Int},Float64}()
for i in 1:n-1, j in i+1:n
    if cluster_of[i] != cluster_of[j]
        mu_ij[(i,j)] = 0.0
    end
end

println("Instance 100 vars   •  b = ", b)


Instance 100 vars   •  b = 64.0


In [57]:
X0, fbar = greedy_linear(c, Q, a, b)
println("greedy_linear fbar = ", fbar)

greedy_linear fbar = 3460.0


In [58]:
X1, fbar1 = greedy_plus_oneflip(c, Q, a, b)
println("greedy_plus_oneflip fbar = ", fbar1)

greedy_plus_oneflip fbar = 3460.0


In [59]:
X_greedy, fbar_greedy = greedy_heuristic(c, Q, a, b)
println("greedy_heuristic fbar = ", fbar_greedy)

greedy_heuristic fbar = 5243.4


In [60]:
X_sa, fbar_sa = simulated_annealing(c, Q, a, b, X_greedy)
println("Simulated Annealing fbar = ", fbar_sa)

Simulated Annealing fbar = 5243.4


In [61]:
ub = compute_first_ub(c, Q, a, b)

5210.0

In [62]:
lamda_opt_m2, mu_opt_m2, dual_val_m2 = subgradient_solve_m2(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 50,
    alpha0 = 1.0,
    p      = 0.9
)

(Dict((10, 79) => 0.0, (13, 66) => 0.0, (17, 12) => 0.0, (19, 86) => 0.0, (9, 51) => 0.0, (19, 14) => 0.0, (9, 80) => 0.0, (16, 99) => 0.0, (19, 16) => 0.0, (12, 45) => 0.0…), Dict((19, 86) => 0.0, (65, 69) => 0.0, (9, 80) => 0.0, (16, 99) => 0.0, (26, 53) => 0.0, (30, 68) => 0.0, (48, 61) => 0.0, (36, 73) => 0.0, (73, 85) => 0.0, (64, 82) => 0.0…), 5169.2)

In [63]:
lamda_opt_m3, mu_opt_m3, dual_val_m3 = subgradient_solve_m3(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    fbar = fbar_sa,
    maxiter = 50,
    p      = 0.9
)



(Dict((10, 79) => 0.0, (13, 66) => 0.0, (17, 12) => 0.0, (19, 86) => 0.0, (9, 51) => 0.0, (19, 14) => 0.0, (9, 80) => 0.0, (16, 99) => 0.0, (19, 16) => 0.0, (12, 45) => 0.0…), Dict((19, 86) => 0.0, (65, 69) => 0.0, (9, 80) => 0.0, (16, 99) => 0.0, (26, 53) => 0.0, (30, 68) => 0.0, (48, 61) => 0.0, (36, 73) => 0.0, (73, 85) => 0.0, (64, 82) => 0.0…), 5169.2)

In [64]:
println("Upper bound initiale: ", ub)
println("Dual bound (méthode 2) = ", dual_val_m2)
println("Dual bound (méthode 3) = ", dual_val_m3)

Upper bound initiale: 5210.0
Dual bound (méthode 2) = 5169.2
Dual bound (méthode 3) = 5169.2


### Exemple 4: n = 10

In [66]:
c = [1.0, 2.0 ,3.0 ,4.0 ,5.0 , 6.0 ,7.0 ,8.0 ,9.0 ,10.0]
Q = zeros(10,10)
Q[1,2] = Q[2,1] = 1
Q[2,3] = Q[3,2] = 1
Q[3,4] = Q[4,3] = 1
Q[4,5] = Q[5,4] = 1
Q[6,7] = Q[7,6] = 2
Q[7,8] = Q[8,7] = 2
Q[8,9] = Q[9,8] = 2
Q[9,10]= Q[10,9]= 2

a = fill(2.0, 10)
cluster_size = 5

# 2) Para cada cluster, elegimos las dos primeras variables y las ponemos a 1
clusters = create_clusters(10, 5)
for Ik in clusters
    # por ejemplo, fijamos Ik[1] e Ik[2] a peso 1
    a[ Ik[1] ] = 1
    a[ Ik[2] ] = 1
end


b = 4.0

cluster_of = Dict{Int,Int}()
pos_in_cluster = Dict{Int,Int}()
for (k, Ik) in enumerate(clusters)
    for (pos, i) in enumerate(Ik)
        cluster_of[i] = k
        pos_in_cluster[i] = pos
    end
end

lambda_kj = Dict{Tuple{Int,Int},Float64}()
for (k, Ik) in enumerate(clusters)
    for j in setdiff(1:length(c), Ik)
        lambda_kj[(k,j)] = 0.0
    end
end

# initialize mu_{i,j} = 0 for every cross-cluster i<j
mu_ij = Dict{Tuple{Int,Int},Float64}()
for i in 1:length(c)-1, j in i+1:length(c)
    if cluster_of[i] != cluster_of[j]
        mu_ij[(i,j)] = 0.0
    end
end
b

4.0

In [32]:
X0, fbar = greedy_linear(c, Q, a, b)
println("greedy_linear fbar = ", fbar)

greedy_linear fbar = 25.0


In [33]:
X1, fbar1 = greedy_plus_oneflip(c, Q, a, b)
println("greedy_plus_oneflip fbar = ", fbar1)

greedy_plus_oneflip fbar = 25.0


In [34]:
X_greedy, fbar_greedy = greedy_heuristic(c, Q, a, b)
println("greedy_heuristic fbar = ", fbar_greedy)

greedy_heuristic fbar = 60.0


In [35]:
X_sa, fbar_sa = simulated_annealing(c, Q, a, b, X_greedy)
println("Simulated Annealing fbar = ", fbar_sa)

Simulated Annealing fbar = 60.0


In [36]:
ub = compute_first_ub(c, Q, a, b)

35.0

In [37]:
lamda_opt_m2, mu_opt_m2, dual_val_m2 = subgradient_solve_m2(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 50,
    alpha0 = 1.0,
    p      = 0.9
)

(Dict((2, 4) => 0.0, (2, 5) => 0.0, (1, 7) => 0.0, (1, 6) => 0.0, (1, 9) => 0.0, (2, 1) => 0.0, (2, 2) => 0.0, (1, 8) => 0.0, (2, 3) => 0.0, (1, 10) => 0.0…), Dict((1, 10) => 0.0, (3, 7) => 0.0, (4, 6) => 0.0, (3, 8) => 0.0, (5, 9) => 0.0, (2, 6) => 0.0, (3, 10) => 0.0, (4, 7) => 0.0, (1, 9) => 0.0, (5, 6) => 0.0…), 33.5)

In [38]:
lamda_opt_m3, mu_opt_m3, dual_val_m3 = subgradient_solve_m3(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    fbar = fbar_sa,
    maxiter = 50,
    p      = 0.9
)



(Dict((2, 4) => 0.0, (2, 5) => 0.0, (1, 7) => 0.0, (1, 6) => 0.0, (1, 9) => 0.0, (2, 1) => 0.0, (2, 2) => 0.0, (1, 8) => 0.0, (2, 3) => 0.0, (1, 10) => 0.0…), Dict((1, 10) => 0.0, (3, 7) => 0.0, (4, 6) => 0.0, (3, 8) => 0.0, (5, 9) => 0.0, (2, 6) => 0.0, (3, 10) => 0.0, (4, 7) => 0.0, (1, 9) => 0.0, (5, 6) => 0.0…), 33.5)

In [39]:
println("Upper bound initiale: ", ub)
println("Dual bound (méthode 2) = ", dual_val_m2)
println("Dual bound (méthode 3) = ", dual_val_m3)

Upper bound initiale: 35.0
Dual bound (méthode 2) = 33.5
Dual bound (méthode 3) = 33.5


### Exemple 5: n = 30

In [40]:
using Random

# 1) Données du problème à 30 variables
n = 30
c = Float64.(1:n)            # [1.0, 2.0, …, 30.0]

# 2) Matrice de couplages Q en « chaînes par blocs »
Q = zeros(n, n)
for i in 1:n-1
    w = i ≤ 10  ? 1.0 :
        i ≤ 20  ? 2.0 :
                  3.0
    Q[i, i+1] = w
    Q[i+1, i] = w
end
# quelques croisements additionnels
Q[5,15]  = Q[15,5]  = 0.5
Q[10,20] = Q[20,10] = 0.7
Q[12,25] = Q[25,12] = 1.2

# 3) Poids a : tous à 2.0, puis on fixe 2 poids à 1 par cluster
a = fill(2.0, n)
cluster_size = 5

# 4) Création des clusters et choix aléatoire de deux indices à poids=1
clusters = create_clusters(n, cluster_size)
for Ik in clusters
    # por ejemplo, fijamos Ik[1] e Ik[2] a peso 1
    a[ Ik[1] ] = 1
    a[ Ik[2] ] = 1
end

# 5) Capacité b (par exemple 40% de la somme des a)
b = 0.4 * sum(a)

# 6) Tables de mapping pour les clusters
cluster_of     = Dict{Int,Int}()
pos_in_cluster = Dict{Int,Int}()
for (k, Ik) in enumerate(clusters)
    for (pos, i) in enumerate(Ik)
        cluster_of[i]     = k
        pos_in_cluster[i] = pos
    end
end

# 7) Initialisation des multiplicateurs λ_{k,j}
lambda_kj = Dict{Tuple{Int,Int},Float64}()
for (k, Ik) in enumerate(clusters)
    for j in setdiff(1:n, Ik)
        lambda_kj[(k,j)] = 0.0
    end
end

# 8) Initialisation des multiplicateurs μ_{i,j}
mu_ij = Dict{Tuple{Int,Int},Float64}()
for i in 1:n-1, j in i+1:n
    if cluster_of[i] != cluster_of[j]
        mu_ij[(i,j)] = 0.0
    end
end

# 9) Affichage de l’instance
println("n            = ", n)
println("cluster_size = ", cluster_size)
println("b            = ", b)
println("clusters     = ", clusters)
println("c            = ", c)
println("a            = ", a)


n            = 30
cluster_size = 5
b            = 19.200000000000003
clusters     = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25], [26, 27, 28, 29, 30]]
c            = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]
a            = [1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0]


In [41]:
X0, fbar = greedy_linear(c, Q, a, b)
println("greedy_linear fbar = ", fbar)

greedy_linear fbar = 330.2


In [42]:
X1, fbar1 = greedy_plus_oneflip(c, Q, a, b)
println("greedy_plus_oneflip fbar = ", fbar1)

greedy_plus_oneflip fbar = 330.2


In [43]:
X_greedy, fbar_greedy = greedy_heuristic(c, Q, a, b)
println("greedy_heuristic fbar = ", fbar_greedy)

greedy_heuristic fbar = 520.4


In [44]:
X_sa, fbar_sa = simulated_annealing(c, Q, a, b, X_greedy)
println("Simulated Annealing fbar = ", fbar_sa)

Simulated Annealing fbar = 520.4


In [49]:
ub = compute_first_ub(c, Q, a, b)

513.0

In [46]:
lamda_opt_m2, mu_opt_m2, dual_val_m2 = subgradient_solve_m2(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    maxiter = 50,
    alpha0 = 1.0,
    p      = 0.9
)

(Dict((1, 28) => 0.0, (3, 23) => 0.0, (1, 12) => 0.0, (3, 29) => 0.0, (1, 15) => 0.0, (6, 21) => 0.0, (4, 6) => 0.0, (3, 28) => 0.0, (5, 5) => 0.0, (2, 22) => 0.0…), Dict((1, 28) => 0.0, (11, 17) => 0.0, (22, 28) => 0.0, (23, 27) => 0.0, (8, 15) => 0.0, (7, 18) => 0.0, (7, 24) => 0.0, (1, 9) => 0.0, (15, 23) => 0.0, (15, 29) => 0.0…), 500.4)

In [47]:
lamda_opt_m3, mu_opt_m3, dual_val_m3 = subgradient_solve_m3(
    Q,       # Matrix{Float64}
    c,     # Vector{Float64}
    a,       # Vector{Float64}
    b,       # Float64
    cluster_size,
    lambda_kj,
    mu_ij;
    fbar = fbar_sa,
    maxiter = 50,
    p      = 0.9
)



(Dict((1, 28) => 0.0, (3, 23) => 0.0, (1, 12) => 0.0, (3, 29) => 0.0, (1, 15) => 0.0, (6, 21) => 0.0, (4, 6) => 0.0, (3, 28) => 0.0, (5, 5) => 0.0, (2, 22) => 0.0…), Dict((1, 28) => 0.0, (11, 17) => 0.0, (22, 28) => 0.0, (23, 27) => 0.0, (8, 15) => 0.0, (7, 18) => 0.0, (7, 24) => 0.0, (1, 9) => 0.0, (15, 23) => 0.0, (15, 29) => 0.0…), 500.4)

In [48]:
println("Upper bound initiale: ", ub)
println("Dual bound (méthode 2) = ", dual_val_m2)
println("Dual bound (méthode 3) = ", dual_val_m3)

Upper bound initiale: 513.0
Dual bound (méthode 2) = 500.4
Dual bound (méthode 3) = 500.4
